In [ ]:
%load_ext dotenv
%dotenv ../redbox/.env.test

In [ ]:
from langgraph.prebuilt import create_react_agent
from redbox.models.chain import ChatLLMBackend
from redbox.loader.loaders import get_chat_llm
from langchain_mcp_adapters.client import MultiServerMCPClient
from redbox.models.settings import Settings

env = Settings()
model = get_chat_llm(ChatLLMBackend(name="anthropic.claude-3-sonnet-20240229-v1:0", provider="bedrock"))

### MCP client via Streamable HTTP

In [ ]:
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from langchain_mcp_adapters.tools import load_mcp_tools

async with streamablehttp_client("http://localhost:1234/mcp/") as (read, write, _):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)
        agent = create_react_agent(model, tools)
        response = await agent.ainvoke({"messages": "tell me about Microsoft"})
response

In [ ]:
# using as multiple servers
async with MultiServerMCPClient(
    {
        "caddy": {
            "url": "http://localhost:1234/mcp/",
            "transport": "streamable_http",
        }
    }
) as client:
    agent = create_react_agent(model, client.get_tools())
    result = await agent.ainvoke({"messages": "tell me about Microsoft"})
result

### SSE

Using Caddy MCP as a tool in react agent

In [ ]:
async with MultiServerMCPClient(
    {"caddy": {"url": env.caddy_mcp.url, "transport": "sse", "headers": env.caddy_mcp.secret_tokens}}
) as client:
    agent = create_react_agent(model, client.get_tools())
    result = await agent.ainvoke({"messages": "Search for grants relating to badgers."})

Using Parlex MCP as a tool in react agent

In [ ]:
async with MultiServerMCPClient(
    {"parlex": {"url": env.parlex_mcp.url, "transport": "sse", "headers": env.parlex_mcp.secret_tokens}}
) as client:
    agent = create_react_agent(model, client.get_tools())
    result = await agent.ainvoke({"messages": "Search for Keir Starmer"})